<a href="https://colab.research.google.com/github/YoungriKIM/Using_Colab/blob/main/samsung_ensemble_0117_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np

# 데이터 불러오기
x1_train = np.load('/content/drive/My Drive/colab_data/ensemble_data_ss.npy', allow_pickle=True)[0]
x1_val = np.load('/content/drive/My Drive/colab_data/ensemble_data_ss.npy', allow_pickle=True)[1]
x1_test = np.load('/content/drive/My Drive/colab_data/ensemble_data_ss.npy', allow_pickle=True)[2]
x1_pred = np.load('/content/drive/My Drive/colab_data/ensemble_data_ss.npy', allow_pickle=True)[3]

x2_train = np.load('/content/drive/My Drive/colab_data/ensemble_data_kodex.npy', allow_pickle=True)[0]
y2_train = np.load('/content/drive/My Drive/colab_data/ensemble_data_kodex.npy', allow_pickle=True)[1]
x2_val = np.load('/content/drive/My Drive/colab_data/ensemble_data_kodex.npy', allow_pickle=True)[2]
y2_val = np.load('/content/drive/My Drive/colab_data/ensemble_data_kodex.npy', allow_pickle=True)[3]
x2_test = np.load('/content/drive/My Drive/colab_data/ensemble_data_kodex.npy', allow_pickle=True)[4]
y2_test = np.load('/content/drive/My Drive/colab_data/ensemble_data_kodex.npy', allow_pickle=True)[5]
x2_pred = np.load('/content/drive/My Drive/colab_data/ensemble_data_kodex.npy', allow_pickle=True)[6]

#모델 구성
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Dropout, Conv1D, Flatten, MaxPooling1D, LSTM, GRU, LeakyReLU, concatenate

#모델1
input1 = Input(shape = (x1_train.shape[1], x1_train.shape[2]))
conv1 = Conv1D(filters = 400, kernel_size = 2, strides=1, padding = 'same', activation='relu')(input1)
pool1 = MaxPooling1D(pool_size=2)(conv1)
conv1 = Conv1D(400, 2, padding='same')(pool1)
conv1 = Conv1D(200, 2, padding='same')(conv1)
conv1 = Conv1D(200, 2, padding='same')(conv1)
pool1 = MaxPooling1D(pool_size=2)(conv1)
flat1 = Flatten()(pool1)
dense1 = Dense(16)(flat1)
dense1 = Dense(16)(dense1)
dense1 = Dense(16)(dense1)
dense1 = Dense(4)(dense1)
dense1 = Dense(4)(dense1)

#모델2
input2 = Input(shape = (x2_train.shape[1], x2_train.shape[2]))
conv2 = Conv1D(filters = 400, kernel_size = 2, strides=1, padding = 'same', activation='relu')(input2)
pool2 = MaxPooling1D(pool_size=2)(conv2)
conv2 = Conv1D(200, 2, padding='same')(pool2)
flat2 = Flatten()(conv2)
dense2 = Dense(32)(flat2)
dense2 = Dense(32)(dense2)
dense2 = Dense(16)(dense2)
dense2 = Dense(16)(dense2)
dense2 = Dense(8)(dense2)
dense2 = Dense(4)(dense2)


#모델 병합
merge1 = concatenate([dense1, dense2])
output1 = Dense(2)(merge1)

model = Model(inputs = [input1, input2], outputs = output1)


#3. 컴파일, 핏
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
stop = EarlyStopping(monitor='val_loss', patience=16, mode='min')

modelpath = '/content/drive/My Drive/colab_data/modelcheckpoint/ss_ensemble_{epoch:02d}-{val_loss:08f}.hdf5'
check = ModelCheckpoint(filepath=modelpath, monitor='val_loss', save_best_only=True, mode='auto')

# hist = model.fit(x_train, y_train, epochs=20, batch_size=4, validation_data=(x_val, y_val), verbose=1, callbacks=[stop])#, check])
hist = model.fit([x1_train, x2_train], y2_train, epochs=100, batch_size=2, validation_data=([x1_val, x2_val], y2_val), verbose=2, callbacks=[stop, check])


#4. 평가, 예측
result = model.evaluate([x1_test, x2_test], y2_test, batch_size=2)
print('mse: ', format(result[0], ','))
print('mae: ', format(result[1], ','))

y_pred = model.predict([x1_pred, x2_pred])
print('1/18일, 19일 삼성주식 시가는: ', y_pred, '입니다.')

# mse:  16,082,799.0        4


Epoch 1/100
160/160 - 3s - loss: 617035136.0000 - mae: 16888.4766 - val_loss: 89930320.0000 - val_mae: 6489.3130
Epoch 2/100
160/160 - 1s - loss: 56898976.0000 - mae: 5526.3193 - val_loss: 71628752.0000 - val_mae: 5486.1958
Epoch 3/100
160/160 - 1s - loss: 45168472.0000 - mae: 4823.9399 - val_loss: 83955584.0000 - val_mae: 6450.2217
Epoch 4/100
160/160 - 2s - loss: 45457912.0000 - mae: 4874.6270 - val_loss: 68623480.0000 - val_mae: 5351.8784
Epoch 5/100
160/160 - 2s - loss: 39701128.0000 - mae: 4467.8159 - val_loss: 69836256.0000 - val_mae: 5727.2515
Epoch 6/100
160/160 - 1s - loss: 37156188.0000 - mae: 4463.6807 - val_loss: 60635572.0000 - val_mae: 5187.9932
Epoch 7/100
160/160 - 1s - loss: 33308444.0000 - mae: 4306.6616 - val_loss: 64754248.0000 - val_mae: 5314.8618
Epoch 8/100
160/160 - 2s - loss: 30867338.0000 - mae: 4125.6465 - val_loss: 61648724.0000 - val_mae: 5080.2896
Epoch 9/100
160/160 - 2s - loss: 29629098.0000 - mae: 4140.8193 - val_loss: 47824488.0000 - val_mae: 4387.5220